In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
from tqdm import tqdm
import os

TODO:
- Table with metrics summary (mean + std)
- Create full plot (with multiple subplots)
- Possibly: assert to check ass exp uses same dsets

In [2]:
# Make sure the path is correct, i.e. change according which experiment we want
# root_path = Path('results_atari')
root_path = Path('results')

experiments = sorted([path.stem for path in root_path.iterdir()])
exp_box = [widgets.Checkbox(value=False, description=label, layout=widgets.Layout(width='600px')) for label in experiments]
exp_vbox = widgets.VBox(exp_box)
display(exp_vbox)

In [ ]:
# OUTPUT_FOLDER = root_path / 'paper_3'
# assert not os.path.exists(OUTPUT_FOLDER)
# os.mkdir(OUTPUT_FOLDER), 'OUTPUT FOLDER ALREADY EXISTS! Delete it or change name'

In [3]:
# Load data
metrics_paths = []
for checkbox in exp_box:
    if checkbox.value == True:
        metrics_paths.append((checkbox.description, root_path / checkbox.description / 'run_plot' / 'metrics.csv'))

# Get only exp names
exps_names = [n for n, _ in metrics_paths]

# load dfs and put in a dict with exp name as key
dfs = {name: pd.read_csv(mpath) for name, mpath in metrics_paths}

In [4]:
metrics_plot = dfs[exps_names[0]]
datasets_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['dataset'])]
datasets_vbox = widgets.VBox(children=datasets_checkboxes)
 
metrics_checkboxes = [widgets.Checkbox(value=True, description=label) for label in pd.unique(metrics_plot['metric'])]
metrics_vbox = widgets.VBox(children=metrics_checkboxes)

split_checkboxes = [widgets.Checkbox(value=True, description=label) for label in ['train', 'validation', 'test']]
split_vbox = widgets.VBox(split_checkboxes)


vboxes = [datasets_vbox, metrics_vbox, split_vbox]
output = widgets.HBox(children=vboxes)
display(output)

In [5]:
# Get selected dsets
dsets = [d.description for d in datasets_checkboxes if d.value == True]
# Get selected metrics
metrics = [m.description for m in metrics_checkboxes if m.value == True]
# Get selected splits (train, validation, test)
splits = [c.description for c in split_checkboxes if c.value == True]


vals = []
# Loop over dsets
for dset_name in dsets:
    # Loop over type (train, validation, test)
    for split in ['test']:#splits:
        # Loop over dset
        for exp in exps_names:
            # Get dataframe
            df = dfs[exp]
            # Filted dset
            df = df[df['dataset'] == dset_name]
            # Select split
            df_split = df[df['split'] == split]
            # Create empty row
            row = []
            # Loop over metrics
            for metric_name in metrics:
                # Filter metrics
                df_metric = df_split[df_split['metric'] == metric_name]
#                 print(df_split[df_split['epoch'] == -1])
                df_metric = df_metric[df_metric['epoch'] == -1]
                row.append(df_metric['value'].item())
            vals.append(row)
            
#                 # Create row name
#                 row_name = f"{exp}_{dset_name}_{metric_name}"
#                 # Insert in df with new index
#                 vs.append(df_split['value'].set_axis(row_name, axis=0).rename(metric_name))
#                 values.insert(0, col_name, df_split['value'].set_axis(df_split['epoch'])) # works also
# Merge into a single df
# values = pd.concat(vs)#, axis=1)
# values
df_all = pd.DataFrame(vals, index=exps_names, columns=metrics)
df_all

,loss,accuracy,precision,recall,f1score,specificity,iou,dice,hd
20230507_bbbbd3_load_100k_7,0.787124,0.984753,0.795964,0.909400,0.848909,0.988477,0.737482,0.848909,116.759819
20230512_load_bts4_4,0.783326,0.982258,0.765402,0.898824,0.826765,0.986382,0.704688,0.826765,86.722513
20230512_load_bts5_5,0.788183,0.984494,0.805506,0.884313,0.843072,0.989446,0.728716,0.843072,97.799135


In [6]:
for col_name, values in df_all.iteritems():
    mn = round(values.mean(), 4)
    std = round(values.std(), 4)
#     print(f"{col_name}: {mn} +- {std}")
    print('{:>22}'.format(col_name), '{:>12}'.format(mn), '{:>2}'.format("+-"), '{:2}'.format(std))

                  loss       0.7862 +- 0.0026
              accuracy       0.9838 +- 0.0014
             precision        0.789 +- 0.021
                recall       0.8975 +- 0.0126
               f1score       0.8396 +- 0.0115
           specificity       0.9881 +- 0.0016
                   iou       0.7236 +- 0.017
                  dice       0.8396 +- 0.0115
                    hd     100.4272 +- 15.1901


/tmp/ipykernel_27048/3605444602.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, values in df_all.iteritems():


## Last value

In [7]:
vals = []
# Loop over dsets
for dset_name in dsets:
    # Loop over dset
    for exp in exps_names:
        # Create empty row
        row = []
        cols_names = []
        # Loop over type (train, validation, test)
        for split in splits:
            # Get dataframe
            df = dfs[exp]
            # Filted dset
            df = df[df['dataset'] == dset_name]
            # Select split
            df_split = df[df['split'] == split]
            # Loop over metrics
            for metric_name in metrics:
                # Filter metrics
                df_metric = df_split[df_split['metric'] == metric_name]
                # Consider last
                if split == 'test':
                    df_metric = df_metric.iloc[-2, :]
                else:
                    df_metric = df_metric.iloc[-1, :]
                row.append(df_metric['value'].item())
                cols_names.append(f"{metric_name}_{split}")
        vals.append(row)
            
#                 # Create row name
#                 row_name = f"{exp}_{dset_name}_{metric_name}"
#                 # Insert in df with new index
#                 vs.append(df_split['value'].set_axis(row_name, axis=0).rename(metric_name))
#                 values.insert(0, col_name, df_split['value'].set_axis(df_split['epoch'])) # works also
# Merge into a single df
# values = pd.concat(vs)#, axis=1)
# values
df_all = pd.DataFrame(vals, index=exps_names, columns=cols_names)
df_all

,loss_train,accuracy_train,precision_train,recall_train,f1score_train,specificity_train,iou_train,dice_train,hd_train,loss_validation,...,hd_validation,loss_test,accuracy_test,precision_test,recall_test,f1score_test,specificity_test,iou_test,dice_test,hd_test
20230507_bbbbd3_load_100k_7,0.639885,0.996492,0.966473,0.996517,0.981265,0.996489,0.963219,0.981265,54.766679,0.701947,...,105.183232,0.781951,0.984509,0.790638,0.912840,0.847356,0.988052,0.735141,0.847356,121.422370
20230512_load_bts4_4,0.644319,0.996101,0.973556,0.984448,0.978972,0.997285,0.958810,0.978972,20.689065,0.705162,...,90.390415,0.788440,0.986376,0.833147,0.888747,0.860050,0.991202,0.754462,0.860050,78.708461
20230512_load_bts5_5,0.639880,0.995567,0.966752,0.985815,0.976190,0.996557,0.953488,0.976190,23.157657,0.706566,...,93.774583,0.787784,0.981973,0.770997,0.878092,0.821067,0.987108,0.696450,0.821067,93.320654


In [8]:
fmt = '{:10} {:<20} {:<10} {:<20}'
for col_name, values in df_all.items():
    mn = round(values.mean(), 4)
    std = round(values.std(), 4)
    print('{:>22}'.format(col_name), '{:>12}'.format(mn), '{:>2}'.format("+-"), '{:2}'.format(std))
#     print(fmt.format(col_name, mn, "+-", std))

            loss_train       0.6414 +- 0.0026
        accuracy_train       0.9961 +- 0.0005
       precision_train       0.9689 +- 0.004
          recall_train       0.9889 +- 0.0066
         f1score_train       0.9788 +- 0.0025
     specificity_train       0.9968 +- 0.0004
             iou_train       0.9585 +- 0.0049
            dice_train       0.9788 +- 0.0025
              hd_train      32.8711 +- 19.0022
       loss_validation       0.7046 +- 0.0024
   accuracy_validation       0.9645 +- 0.0014
  precision_validation       0.8031 +- 0.023
     recall_validation       0.7217 +- 0.0587
    f1score_validation       0.7587 +- 0.0216
specificity_validation        0.985 +- 0.0035
        iou_validation       0.6116 +- 0.0282
       dice_validation       0.7587 +- 0.0216
         hd_validation      96.4494 +- 7.7507
             loss_test       0.7861 +- 0.0036
         accuracy_test       0.9843 +- 0.0022
        precision_test       0.7983 +- 0.0318
           recall_test       0.8932

## 